# Question
The following figures show the Di Wang Tower in Shenzhen. The structure of Di Wang Tower is made up of steel and reinforced concrete (calculated as reinforced concrete), C terrain type, the design wind pressure $w_0 = 0.75 \text{ kN/m}^2$, period $T_1=6.38\text{ s}$, The first mode shape is listed in following table. To determine shape factor, the cross section could be considered as rectangle.

<center>
    <img src="./images/Diwang01.png" alt="The Elvation of the DiWang Tower" width="400"/>
    <img src="./images/Diwang02.png" alt="The Plan View of the DiWang Tower" width="400"/>
</center>

<br>

| $\dfrac{h}{H}$ | 0     | 0.1   | 0.2   | 0.3   | 0.4    | 0.5   | 0.6   | 0.7    | 0.8   | 0.9   | 1.0   |
| -------------- | ----- | ----- | ----- | ----- | ------ | ----- | ----- | ------ | ----- | ----- | ----- |
| Mode shape     | 0.0000| 0.0137| 0.0558| 0.1277| 0.2424| 0.3503| 0.4629| 0.5903| 0.7309| 0.8700| 1.0000| 
| Mass / ton     | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 1500  | 

Only consider the first mode, calculate **the extreme of top displacements** induced by the along-wind static and dynamic wind load in the given wind direction using the following methods:
1. Method recommended by the Chinese code for wind-resistant design of buildings (GB 50009-2012).
2. According to the quasi-steady assumption, convert the fluctuating wind velocity time histories in `windData` (**with a duration of 10 minutes and a sampling frequency of 10 Hz**) to wind load time histories $F_i(t)=[1/2\rho \bar{u_i}^2+\rho \bar{u_i} u'_i(t)] \cdot \mu_{s,i} \cdot A_i$, and then using stocastic vibration methods in time domain (such as the newmark-beta algorithm) and in frequency domain (such as the response spectrum analysis). Peak factor $g = 2.5$ should be used for the extreme value calculation, and the programming using frequency domain method is optional.

# Answer


In [16]:
import scipy.io as sio

# load data
simDataPath = './windData/windData.mat'
simData = sio.loadmat(simDataPath, squeeze_me=True, struct_as_record=False)
U = simData['U']
Z = simData['Z']
dt = simData['dt']
t = simData['t']
del simData
print("\nThe dimension of U is:", U.shape)



The dimension of U is: (11, 6000)


In [17]:
#采用国标推荐方法
import numpy as np

# 参数设置
w0 = 0.75  # kN/m²
T1 = 6.38  # s
mu_s = 1.3  # 体型系数
H = 324.95  # 总高度 (m)
alpha = 0.44  # C类地形
xi = 2.0  # 脉动增大系数
nu = 0.4  # 脉动影响系数
g = 2.5  # 峰值因子

# 振型和质量数据
h_over_H = np.array([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
phi = np.array([0.0000, 0.0137, 0.0558, 0.1277, 0.2424, 0.3503, 0.4629, 0.5903, 0.7309, 0.8700, 1.0000])
mass = np.array([2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 1500]) * 1000  # 转换为 kN·s²/m

# 计算各层高度（确保z不小于5米）
z = h_over_H * H
z = np.clip(z, a_min=5, a_max=None)  # 限制最小高度为5米

# 风压高度系数
mu_z = (z / 10) ** alpha

# 风振系数β_z
beta_z = 1 + (xi * nu * phi) / mu_z

# 计算静风荷载
B = 35.5  # 结构宽度 
delta_h = H / 10  # 每层高度段
A_i = B * delta_h  # 迎风面积 (m²)

# 计算静风荷载
F_i = beta_z * mu_s * mu_z * w0 * A_i  # kN

# 广义参数
P_star = np.sum(F_i * phi)
M_star = np.sum(mass * phi**2)
omega1 = 2 * np.pi / T1
K_star = omega1**2 * M_star

# 静力位移
X_static = P_star / K_star  # 顶部静力位移 (m)

print(f"按中国规范计算的顶部静力位移极值: {X_static:.4f} m")

按中国规范计算的顶部静力位移极值: 0.0036 m


In [18]:
#采用Quasi-steady assumption 
from scipy.integrate import odeint

# 加载风速数据
rho = 1.25  # 空气密度 (kg/m³)
mu_s = 1.3  # 体型系数
B = 35.5  # 结构宽度 (假设)
H = Z[-1]  # 从windData.mat中读取Z[-1]为总高度
delta_h = H / 10
A_i = B * delta_h  # 每层迎风面积

# 计算平均风速ū_i和脉动风荷载
ū = (Z / 10)**0.22 * np.sqrt(2 * w0 * 1e3 / rho)  # 转换为m/s
F_time = np.zeros_like(U)
for i in range(11):
    F_static = 0.5 * rho * ū[i]**2 * mu_s * A_i
    F_fluctuating = rho * ū[i] * U[i] * mu_s * A_i
    F_time[i] = F_static + F_fluctuating

# 广义力时程
P_t = np.sum(F_time * phi[:, None], axis=0)

# Newmark-β参数
beta = 0.25
gamma = 0.5
dt = 0.1  # 时间步长
M_star = np.sum(mass * phi**2)
K_star = (2 * np.pi / T1)**2 * M_star
zeta = 0.05  # 阻尼比
C_star = 2 * zeta * np.sqrt(K_star * M_star)

# 初始化
q = np.zeros(len(P_t))
v = np.zeros(len(P_t))
a = np.zeros(len(P_t))
a[0] = (P_t[0] - C_star * v[0] - K_star * q[0]) / M_star

# Newmark-β迭代
for i in range(1, len(P_t)):
    q_pred = q[i-1] + dt * v[i-1] + (0.5 - beta) * dt**2 * a[i-1]
    v_pred = v[i-1] + (1 - gamma) * dt * a[i-1]
    a[i] = (P_t[i] - K_star * q_pred - C_star * v_pred) / (M_star + gamma * dt * C_star + beta * dt**2 * K_star)
    q[i] = q_pred + beta * dt**2 * a[i]
    v[i] = v_pred + gamma * dt * a[i]

# 极端位移
X_extreme = np.max(np.abs(q)) * g
print(f"时域分析顶部极端位移: {X_extreme:.4f} m")

时域分析顶部极端位移: 33.4173 m
